In [ ]:
# Chains in LangChain

## Connect to OpenAI and define our model

Create an OpenAI API key for this project at [https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys). 

Copy `.env.sample` to `.env` and replace `<YOUR_OPENAI_API_KEY>` with the one you created.

Let's get started!

Let's make sure that we can read our OpenAI key from the `.env` file we created.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Specify our model
llm_model="gpt-3.5-turbo-1106"

## Load our example CSV file using pandas

Before we begin, let's install the following dependencies from the command-line of our Python virtual environment:

```sh
(.venv) % pip install pandas
(.venv) % pip freeze > requirements.txt
```

## LLMChain

## Sequential Chains

### SimpleSequentialChain

### SequentialChain

## Router Chain